In [1]:
import numpy as np
from matplotlib import pyplot as plt
import time

from PIL import Image
import cv2

import torch
import torchvision
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms as tfs
torch.manual_seed(42)

import albumentations as A
from albumentations.pytorch import ToTensorV2
from albumentations.augmentations import transforms as AT

import imgaug.augmenters as iaa 

from glob import glob

from sklearn.model_selection import train_test_split


In [ ]:
def ex_al(x):
    degree = np.random.randint(359)
    albumentations_transform = A.Compose([
        # A.Resize(256, 256), 
        # A.Rotate((degree, degree), p=1.0),
        A.CenterCrop(128, 128),
        
        # A.ToGray(p=1),
        # A.ColorJitter (brightness=0.2, contrast=0.2, saturation=0.2, 
        #                 hue=0.2, always_apply=False, p=0.7),
        A.Normalize(
            mean=[0.5, 0.5, 0.5],
            std=[0.5, 0.5, 0.5],
        ),
        A.pytorch.transforms.ToTensorV2()
    ])

    start_t = time.time()
    x = cv2.imread(x)
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = cv2.resize(x, (256, 256), interpolation=cv2.INTER_LANCZOS4)
    augmented = albumentations_transform(image=x) 
    image = augmented['image']
    return image, degree

In [ ]:
exam_path = './data/test100/01_20210519_072020.png'
test_path = './data/test_samples/easyset/3.png'

In [ ]:
img, degree = ex_al(exam_path)
print(degree)
plt.figure(figsize=(5, 5))
plt.imshow(tfs.ToPILImage()(img))

In [ ]:
from PIL import ImageFilter

In [ ]:
def ex_tf(x):
    im_aug = tfs.Compose([
            tfs.CenterCrop(128),
            # tfs.RandomGrayscale(p=0.8),
            # tfs.ColorJitter(brightness=0.2, contrast=0.2, hue=0.2)
            ])
    
    degree = np.random.randint(359) 
    im_rota = tfs.Compose([
        #     tfs.CenterCrop(2048),
        #     tfs.Resize(1024),
        #     tfs.Resize(512),
            # tfs.Resize(256),
            # tfs.RandomRotation(degrees=(degree, degree))
    ])
#     x = x.filter(ImageFilter.DETAIL)
    # x = x.resize((256, 256), Image.LANCZOS)
    
    # x = im_rota(x)
    x = im_aug(x)
    return x, degree

In [ ]:
exam = Image.open(exam_path)
score = Image.open(test_path)

img, y = ex_tf(exam)
print(y)
plt.figure(figsize=(8,8))
plt.imshow(img)

# 데이터 경로 불러오기

In [2]:
device = torch.device('cuda:0')
# mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
# std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()
mean = torch.Tensor([0.5]).cuda().half()
std = torch.Tensor([0.5]).cuda().half()

In [3]:
X = glob('./data/train/*.png')

DATA_PATH_TRAIN_LIST, DATA_PATH_TEST_LIST = train_test_split(X, 
                                                            test_size=0.1, 
                                                            random_state=42)

print(len(DATA_PATH_TRAIN_LIST))
print(len(DATA_PATH_TEST_LIST))

10753
1195


# 토치 비젼 증강

In [4]:
class TrainImageTransform():
    
    def __init__(self):
        self.im_aug = tfs.Compose([
            # tfs.Resize(256),
            tfs.CenterCrop(224),
            # tfs.RandomPerspective(), # 실험3. 입체방향 각도의 인식(param부터 다시 얻어와야할듯)
            # tfs.RandomRotation(360),
            tfs.RandomGrayscale(p=1),
            tfs.ColorJitter(brightness=0.3, contrast=0.3, hue=0.3),
            tfs.ToTensor(),
            tfs.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
            ])

    def __call__(self, img):
        degree = np.random.randint(359) 
        im_rota = tfs.Compose([
                tfs.Resize(512),
                tfs.RandomRotation(degrees=(degree, degree))
        ])
        img = im_rota(img)
        img = self.im_aug(img)
        return img, degree


In [5]:
class TestImageTransform():
    
    def __init__(self):
        self.im_aug = tfs.Compose([
            # tfs.Resize(256),
            tfs.CenterCrop(224),
            # tfs.RandomPerspective() # 실험3. 성능의 차이를 보자 !
            tfs.ToTensor(),
            tfs.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
            ])

    def __call__(self, img):
        degree = np.random.randint(359) 
        im_rota = tfs.Compose([
                tfs.Resize(512),
                tfs.RandomRotation(degrees=(degree, degree))
        ])
        img = im_rota(img)
        img = self.im_aug(img)
        return img, degree

In [6]:
class Img_Dataset(Dataset):
    
    def __init__(self, file_list, transform):
        self.file_list = file_list
        self.transform = transform

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, index):
        img_path = self.file_list[index]
        img = Image.open(img_path).convert('RGB') # 픽셀당 추가 알파 채널이 있다고 생각 하므로 채널이 3개가 아닌 4개입니다.
        img_transformed, angle = self.transform(img)

        return img_transformed, angle

In [7]:
train_val_dataset = Img_Dataset(file_list=DATA_PATH_TRAIN_LIST,
                        transform=TrainImageTransform())

train_dataset, val_dataset = random_split(train_val_dataset, [9000, 1753])

test_dataset = Img_Dataset(file_list=DATA_PATH_TEST_LIST,
                        transform=TestImageTransform())


# 배치마다 다른 각도 부여 (배치사이즈 64, 32 실험)

In [ ]:
class Train_Dataset(Dataset):
    
    def __init__(self, file_list):
        self.file_list = file_list

        self.degree = np.random.randint(359) 
        self.im_aug =  A.Compose([
            A.Resize(256, 256), 
            A.Rotate((self.degree, self.degree), p=1.0),
            A.CenterCrop(128, 128),
            A.ToGray(p=0.6),
            A.ColorJitter (brightness=0.2, contrast=0.2, saturation=0.2, 
                            hue=0.2, always_apply=False, p=0.6
                            ),
            A.Normalize(
                mean=[0.5, 0.5, 0.5],
                std=[0.5, 0.5, 0.5]
                ),
            A.pytorch.transforms.ToTensorV2()
        ])

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, index):
        img_path = self.file_list[index]
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        augmented = self.im_aug(image=image)
        image = augmented['image']

        return image, self.degree 

In [ ]:
class Test_Dataset(Dataset):
    
    def __init__(self, file_list):
        self.file_list = file_list

        self.degree = np.random.randint(359) 
        self.im_aug =  A.Compose([
            A.Resize(256, 256), 
            A.Rotate((self.degree, self.degree), p=1.0),
            A.RandomCrop(128, 128),
            A.Normalize(
                mean=[0.5, 0.5, 0.5],
                std=[0.5, 0.5, 0.5]
                ),
            A.pytorch.transforms.ToTensorV2()
        ])

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, index):
        img_path = self.file_list[index]
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        augmented = self.im_aug(image=image)
        image = augmented['image']
        
        return image, self.degree 

# 각 데이터마다 다른 각도 부여

In [ ]:
class TrainDataset(Dataset):
    def __init__(self, file_paths):
        self.file_paths = file_paths

        self.im_aug =  A.Compose([
            A.Resize(256, 256), 
            A.CenterCrop(128, 128),
            A.ToGray(p=0.6),
            A.ColorJitter (brightness=0.2, contrast=0.2, saturation=0.2, 
                            hue=0.2, always_apply=False, p=0.6
                            ),
            A.Normalize(
                mean=[0.5, 0.5, 0.5],
                std=[0.5, 0.5, 0.5]
                ),
            ToTensorV2()
        ])

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        file_paths = self.file_paths[idx]

        image = cv2.imread(file_paths)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        degree = np.random.randint(359)
        al_rotate = A.Compose([
                        A.Rotate((degree, degree), p=1.0)
        ])
        image = al_rotate(image=image)
        image = image['image']

        augmented = self.im_aug(image=image)
        image = augmented['image']

        return image, degree

In [ ]:
class TestDataset(Dataset):
    def __init__(self, file_paths):
        self.file_paths = file_paths

        self.im_aug =  A.Compose([
            A.Resize(256, 256), 
            A.RandomCrop(128, 128), ######################## 랜덤 자르기 !!
            A.Normalize(
                mean=[0.5, 0.5, 0.5],
                std=[0.5, 0.5, 0.5]
                ),
            ToTensorV2()
        ])

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        file_paths = self.file_paths[idx]

        image = cv2.imread(file_paths)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        degree = np.random.randint(359)
        al_rotate = A.Compose([
                        A.Rotate((degree, degree), p=1.0)
        ])
        image = al_rotate(image=image)
        image = image['image']

        augmented = self.im_aug(image=image)
        image = augmented['image']

        return image, degree

# Data Loder

In [8]:
# train_val_dataset = TrainDataset(file_paths=DATA_PATH_TRAIN_LIST)

# train_dataset, val_dataset = random_split(train_val_dataset, [9000, 1753])

train_dataloader = DataLoader(train_dataset,
                            batch_size=32, # 실험4. 성능 그래프 확인 요
                            shuffle=True)

val_dataloader = DataLoader(val_dataset,
                            batch_size=32, # 실험4. 성능 그래프 확인 요
                            shuffle=True)

# test_dataset = TestDataset(file_paths=DATA_PATH_TEST_LIST)

test_dataloader = DataLoader(test_dataset,
                            batch_size=32,
                            shuffle=True)

batch_iterator = iter(train_dataloader)
images = next(batch_iterator)

print(images[0].size())

torch.Size([32, 3, 224, 224])


In [9]:
images[1]

tensor([164,   5, 144, 178, 205,  79,  40, 349, 226, 178, 221,  25, 220, 105,
         44, 176,  87, 282, 341,  40, 102,  38, 106, 321,  95, 200, 187,  12,
        194, 271, 317, 257])

In [10]:
partition = {'train': train_dataloader, 'val':val_dataloader, 'test':test_dataloader}

In [11]:
cfg = {
    'VGG1M256': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M', 1024, 1024, 'M'],
    'VGG11': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG13': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'VGG1M16': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 512, 'M', 1024, 1024, 'M', 1024, 1024, 'M'],
}

In [ ]:
class CNN(nn.Module):
    
    def __init__(self, model_code, in_channels, out_dim, act, use_bn):
        super(CNN, self).__init__()
        
        if act == 'relu':
            self.act = nn.ReLU()
        elif act == 'sigmoid':
            self.act = nn.Sigmoid()
        elif act == 'tanh':
            self.act = nn.TanH()
        else:
            raise ValueError("Not a valid activation function code")
        
        self.layers = self._make_layers(model_code, in_channels, use_bn)
        self.fcn = nn.Sequential(nn.Conv2d(in_channels=7*7*512,
                                           out_channels=4096,
                                           kernel_size=1),
                                 )
        self.classifer = nn.Sequential(nn.Linear(4096, 4096),
                                       self.act,
                                       nn.Linear(4096, out_dim))
        
    def forward(self, x):
        x = self.layers(x)
        x = x.view(x.size(0), -1)
        x = self.classifer(x)
        return x
        
    def _make_layers(self, model_code, in_channels, use_bn):
        layers = []
        for x in cfg[model_code]:
            if x == 'M':
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            else:
                layers += [nn.Conv2d(in_channels=in_channels,
                                     out_channels=x,
                                     kernel_size=3,
                                     stride=1,
                                     padding=1)]
                if use_bn:
                    layers += [nn.BatchNorm2d(x)]
                layers += [self.act]
                in_channels = x
        return nn.Sequential(*layers)

In [ ]:
def dim_check():
    net = CNN('VGG13', 3, 360, 'relu', True)
    x = torch.randn(64, 3, 224, 224)
    y = net(x)
    print(y.size())

In [ ]:
# dim_check()

In [ ]:
def train(net, partition, optimizer, criterion, args):
    net.train()

    correct = 0
    total = 0
    train_loss = 0.0
    for i, data in enumerate(partition['train'], 0):
        optimizer.zero_grad() # [21.01.05 오류 수정] 매 Epoch 마다 .zero_grad()가 실행되는 것을 매 iteration 마다 실행되도록 수정했습니다. 

        # get the inputs
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = net(inputs)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_loss = train_loss / len(partition['train'])
    train_acc = 100 * correct / total
    return net, train_loss, train_acc

In [ ]:
def validate(net, partition, criterion, args):
    net.eval()

    correct = 0
    total = 0
    val_loss = 0 
    with torch.no_grad():
        for data in partition['val']:
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            outputs = net(images)

            loss = criterion(outputs, labels)
            
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        val_loss = val_loss / len(partition['val'])
        val_acc = 100 * correct / total
    return val_loss, val_acc

In [ ]:
def test(net, partition, args):
    
    net.eval()
    
    correct = 0
    total = 0
    with torch.no_grad():
        for data in partition['test']:
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)

            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        test_acc = 100 * correct / total
    return test_acc

In [ ]:
def experiment(partition, args):
      
    net = CNN(model_code = args.model_code,
              in_channels = args.in_channels,
              out_dim = args.out_dim,
              act = args.act,
              use_bn = args.use_bn)
    net.to(device)

    criterion = nn.CrossEntropyLoss()
    
    if args.optim == 'SGD':
        optimizer = optim.SGD(net.parameters(), lr=args.lr, weight_decay=args.l2)
    elif args.optim == 'RMSprop':
        optimizer = optim.RMSprop(net.parameters(), lr=args.lr, weight_decay=args.l2)
    elif args.optim == 'Adam':
        optimizer = optim.Adam(net.parameters(), lr=args.lr, weight_decay=args.l2)
    else:
        raise ValueError('In-valid optimizer choice')
    
    train_losses = []
    val_losses = []
    train_accs = []
    val_accs = []
        
    for epoch in range(args.epoch):  # loop over the dataset multiple times
        ts = time.time()
        net, train_loss, train_acc = train(net, partition, optimizer, criterion, args)
        val_loss, val_acc = validate(net, partition, criterion, args)
        te = time.time()
        
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        train_accs.append(train_acc)
        val_accs.append(val_acc)
        
        print('Epoch {}, Acc(train/val): {:2.2f}/{:2.2f}, Loss(train/val) {:2.2f}/{:2.2f}. Took {:2.2f} sec'.format(epoch, train_acc, val_acc, train_loss, val_loss, te-ts))
        
    test_acc = test(net, partition, args)    
    
    result = {}
    result['train_losses'] = train_losses
    result['val_losses'] = val_losses
    result['train_accs'] = train_accs
    result['val_accs'] = val_accs
    result['train_acc'] = train_acc
    result['val_acc'] = val_acc
    result['test_acc'] = test_acc
    return vars(args), result

In [ ]:
import hashlib
import json
from os import listdir
from os.path import isfile, join
import pandas as pd

!mkdir results

def save_exp_result(setting, result):
    exp_name = setting['exp_name']
    del setting['epoch']
    del setting['test_batch_size']

    hash_key = hashlib.sha1(str(setting).encode()).hexdigest()[:6]
    filename = './results/{}-{}.json'.format(exp_name, hash_key)
    result.update(setting)
    with open(filename, 'w') as f:
        json.dump(result, f)

    
def load_exp_result(exp_name):
    dir_path = './results'
    filenames = [f for f in listdir(dir_path) if isfile(join(dir_path, f)) if '.json' in f]
    list_result = []
    for filename in filenames:
        if exp_name in filename:
            with open(join(dir_path, filename), 'r') as infile:
                results = json.load(infile)
                list_result.append(results)
    df = pd.DataFrame(list_result) # .drop(columns=[])
    return df

In [ ]:
import seaborn as sns
import argparse
from copy import deepcopy

In [ ]:
def plot_acc(var1, var2, df):
    
    fig, ax = plt.subplots(1, 3)
    fig.set_size_inches(15, 6)
    sns.set_style("darkgrid", {"axes.facecolor": ".9"})

    sns.barplot(x=var1, y='train_acc', hue=var2, data=df, ax=ax[0])
    sns.barplot(x=var1, y='val_acc', hue=var2, data=df, ax=ax[1])
    sns.barplot(x=var1, y='test_acc', hue=var2, data=df, ax=ax[2])
    
    ax[0].set_title('Train Accuracy')
    ax[1].set_title('Validation Accuracy')
    ax[2].set_title('Test Accuracy')

    
def plot_loss_variation(var1, var2, df, **kwargs):

    list_v1 = df[var1].unique()
    list_v2 = df[var2].unique()
    list_data = []

    for value1 in list_v1:
        for value2 in list_v2:
            row = df.loc[df[var1]==value1]
            row = row.loc[df[var2]==value2]

            train_losses = list(row.train_losses)[0]
            val_losses = list(row.val_losses)[0]

            for epoch, train_loss in enumerate(train_losses):
                list_data.append({'type':'train', 'loss':train_loss, 'epoch':epoch, var1:value1, var2:value2})
            for epoch, val_loss in enumerate(val_losses):
                list_data.append({'type':'val', 'loss':val_loss, 'epoch':epoch, var1:value1, var2:value2})

    df = pd.DataFrame(list_data)
    g = sns.FacetGrid(df, row=var2, col=var1, hue='type', **kwargs)
    g = g.map(plt.plot, 'epoch', 'loss', marker='.')
    g.add_legend()
    g.fig.suptitle('Train loss vs Val loss')
    plt.subplots_adjust(top=0.89) # 만약 Title이 그래프랑 겹친다면 top 값을 조정해주면 됩니다! 함수 인자로 받으면 그래프마다 조절할 수 있겠죠?


def plot_acc_variation(var1, var2, df, **kwargs):
    list_v1 = df[var1].unique()
    list_v2 = df[var2].unique()
    list_data = []

    for value1 in list_v1:
        for value2 in list_v2:
            row = df.loc[df[var1]==value1]
            row = row.loc[df[var2]==value2]

            train_accs = list(row.train_accs)[0]
            val_accs = list(row.val_accs)[0]
            test_acc = list(row.test_acc)[0]

            for epoch, train_acc in enumerate(train_accs):
                list_data.append({'type':'train', 'Acc':train_acc, 'test_acc':test_acc, 'epoch':epoch, var1:value1, var2:value2})
            for epoch, val_acc in enumerate(val_accs):
                list_data.append({'type':'val', 'Acc':val_acc, 'test_acc':test_acc, 'epoch':epoch, var1:value1, var2:value2})

    df = pd.DataFrame(list_data)
    g = sns.FacetGrid(df, row=var2, col=var1, hue='type', **kwargs)
    g = g.map(plt.plot, 'epoch', 'Acc', marker='.')

    def show_acc(x, y, metric, **kwargs):
        plt.scatter(x, y, alpha=0.3, s=1)
        metric = "Test Acc: {:1.3f}".format(list(metric.values)[0])
        plt.text(0.05, 0.95, metric,  horizontalalignment='left', verticalalignment='center', transform=plt.gca().transAxes, bbox=dict(facecolor='yellow', alpha=0.5, boxstyle="round,pad=0.1"))
    g = g.map(show_acc, 'epoch', 'Acc', 'test_acc')

    g.add_legend()
    g.fig.suptitle('Train Accuracy vs Val Accuracy')
    plt.subplots_adjust(top=0.89)

In [ ]:
# ====== Random seed Initialization ====== #
seed = 123
np.random.seed(seed)
torch.manual_seed(seed)

parser = argparse.ArgumentParser()
args = parser.parse_args("")
args.exp_name = "exp1_lr_model_code"

# ====== Model ====== #
args.model_code = 'VGG13'
args.in_channels = 3
args.out_dim = 360
args.act = 'relu'

# ====== Regularization ======= #
args.l2 = 0.00001
args.use_bn = True

# ====== Optimizer & Training ====== #
args.optim = 'Adam' #'RMSprop' #SGD, RMSprop, ADAM...
args.lr = 0.0001
args.epoch = 50

args.train_batch_size = 32
args.test_batch_size = 32

# ====== Experiment Variable ====== #
name_var1 = 'lr'
name_var2 = 'model_code'
list_var1 = [0.0001]
list_var2 = ['VGG13']


for var1 in list_var1:
    for var2 in list_var2:
        setattr(args, name_var1, var1)
        setattr(args, name_var2, var2)
        print(args)
                
        setting, result = experiment(partition, deepcopy(args))
        # save_exp_result(setting, result)

In [ ]:
ar1 = 'lr'
var2 = 'optim'
df = load_exp_result('exp1')

plot_acc(var1, var2, df)
plot_loss_variation(var1, var2, df, sharey=False) 
#sharey를 True로 하면 모둔 subplot의 y축의 스케일이 같아집니다.
plot_acc_variation(var1, var2, df, margin_titles=True, sharey=True) 
#margin_titles를 True로 하면 그래프의 가장자리에 var1과 var2 값이 표시되고 False로 하면 각 subplot 위에 표시됩니다.

In [ ]:
!nvidia-smi

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

model = CNN('VGG11', 3, 360, 'relu', True)
model.to(device)

model.load_state_dict(torch.load('0001-11-50.pt'))

In [ ]:
test_acc, current_labels, current_preds = test(model, partition, args)    

result_test = {}
result_test['test_acc'] = test_acc
result_test['test_labels'] = current_labels
result_test['test_preds'] = current_preds

In [3]:
from yolov5.lomin import main, parse_opt

promp = ['--source', './data/test_samples/pro/', '--weights', './yolov5/runs/train/doc2007/weights/best.pt', '--save-crop']
opt = parse_opt(promp)
crop_img_list = main(opt)

lomin: weights=['./yolov5/runs/train/doc2007/weights/best.pt'], source=./data/test_samples/pro/, data=yolov5\data\coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=True, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5\runs\detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5  v6.1-21-ge6e36aa torch 1.7.1+cu110 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)

Fusing layers... 
Model Summary: 367 layers, 46108278 parameters, 0 gradients, 107.8 GFLOPs
image 1/27 D:\lomin\data\test_samples\pro\KakaoTalk_20211109_131112737.jpg: 640x480 1 Doc, Done. (0.320s)
image 2/27 D:\lomin\data\test_samples\pro\KakaoTalk_20211109_133751394.jpg: 480x640 Done. (0.315s)
image 3/27 D:\lomin\data\test_samples\pro\KakaoTalk_20211109_133751394_01.jpg: 640x480 Done. (0.209s)
image 4/27 D:

In [18]:
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as img

In [42]:
print(len(crop_img_list))
print(len(crop_img_list[0]))
print(len(crop_img_list[0][0]))
print(len(crop_img_list[0][0][0]))

1
1077
1430
3


In [50]:
a = { 0 : 10, 1 : 20, 2:30, 3:4}
a

{0: 10, 1: 20, 2: 30, 3: 4}

In [57]:
max(a, key=a.get)

2